In [3]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [4]:
data_train, meta = scipy.io.arff.loadarff('yeast-train.arff')
df_train = pd.DataFrame(data_train)
data_test, meta = scipy.io.arff.loadarff('yeast-test.arff')
df_test = pd.DataFrame(data_test)
df_test.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,b'0',b'0',b'1',b'1',b'0',b'0',b'0',b'1',b'1',b'0'
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'0'
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,b'1',b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'


In [5]:
X_train = df_train.iloc[:,:-14].values
y_train = df_train.iloc[:,-14:].values.astype(int)
X_test = df_test.iloc[:,:-14].values
y_test = df_test.iloc[:,-14:].values.astype(int)

In [6]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1500, 103)
Train_Y:  (1500, 14)
Test_X:  (917, 103)
Test_Y:  (917, 14)


In [7]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [8]:
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [9]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='first'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='last'))
    model.add(Dropout(0.4))    
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='output'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

def create_copynode(X_feed,y_now,w_name):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='copyfirst'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='copylast'))
    model.add(Dropout(0.4)) 
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='copyoutput'))
    model.load_weights(w_name,by_name=True)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [10]:
from time import time

In [11]:
#Model chain
chain = []
first = 1
#Training
X_feed = X_train.copy()
t1 = time()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))
    if first == 1:
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=20,batch_size=50)
        '''
        json_fl = node.to_json()
        with open('node1.json','w') as f:
            f.write(json_fl)
        '''
        node.save_weights('weights.h5')
        first = 0
    else:
        node = create_copynode(X_feed,y_now,'weights.h5')
        node.fit(X_feed,y_now,epochs=5,batch_size=50)
        
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)
t2 = time()

print("Time Taken: ",(t2-t1))

Training chain node  0
Shapes:
 X = (1500, 103) 
 Y = (1500, 1)
Epoch 1/20
1500/1500 [==============================] - 15s 10ms/step - loss: 0.6163 - acc: 0.6773
Epoch 2/20
1500/1500 [==============================] - 0s 176us/step - loss: 0.5392 - acc: 0.7007
Epoch 3/20
1500/1500 [==============================] - 0s 180us/step - loss: 0.4877 - acc: 0.7873
Epoch 4/20
1500/1500 [==============================] - 0s 183us/step - loss: 0.4465 - acc: 0.8080
Epoch 5/20
1500/1500 [==============================] - 0s 178us/step - loss: 0.4019 - acc: 0.8327
Epoch 6/20
1500/1500 [==============================] - 0s 189us/step - loss: 0.3952 - acc: 0.8327
Epoch 7/20
1500/1500 [==============================] - 0s 173us/step - loss: 0.3321 - acc: 0.8667
Epoch 8/20
1500/1500 [==============================] - 0s 183us/step - loss: 0.3158 - acc: 0.8727
Epoch 9/20
1500/1500 [==============================] - 0s 175us/step - loss: 0.2649 - acc: 0.8940
Epoch 10/20
1500/1500 [======================

1500/1500 [==============================] - 0s 140us/step - loss: 0.5241 - acc: 0.7500
Epoch 3/5
1500/1500 [==============================] - 0s 102us/step - loss: 0.4893 - acc: 0.7513
Epoch 4/5
1500/1500 [==============================] - 0s 102us/step - loss: 0.4680 - acc: 0.7600
Epoch 5/5
1500/1500 [==============================] - 0s 106us/step - loss: 0.4276 - acc: 0.7980
Training of node 11 complete


Training chain node  12
Shapes:
 X = (1500, 115) 
 Y = (1500, 1)
Epoch 1/5
1500/1500 [==============================] - 1s 926us/step - loss: 0.5828 - acc: 0.6733
Epoch 2/5
1500/1500 [==============================] - 0s 142us/step - loss: 0.2453 - acc: 0.8827
Epoch 3/5
1500/1500 [==============================] - 0s 130us/step - loss: 0.0535 - acc: 0.9920
Epoch 4/5
1500/1500 [==============================] - 0s 122us/step - loss: 0.0293 - acc: 0.9947
Epoch 5/5
1500/1500 [==============================] - 0s 118us/step - loss: 0.0159 - acc: 0.9960
Training of node 12 complete


T

TypeError: 'str' object is not callable

In [12]:
print("Time Taken: ",(t2-t1))

Time Taken:  64.96641302108765


In [13]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (917, 103)
Getting op from chain node  1
Shapes:
 X = (917, 104)
Getting op from chain node  2
Shapes:
 X = (917, 105)
Getting op from chain node  3
Shapes:
 X = (917, 106)
Getting op from chain node  4
Shapes:
 X = (917, 107)
Getting op from chain node  5
Shapes:
 X = (917, 108)
Getting op from chain node  6
Shapes:
 X = (917, 109)
Getting op from chain node  7
Shapes:
 X = (917, 110)
Getting op from chain node  8
Shapes:
 X = (917, 111)
Getting op from chain node  9
Shapes:
 X = (917, 112)
Getting op from chain node  10
Shapes:
 X = (917, 113)
Getting op from chain node  11
Shapes:
 X = (917, 114)
Getting op from chain node  12
Shapes:
 X = (917, 115)
Getting op from chain node  13
Shapes:
 X = (917, 116)


In [14]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [15]:
from sklearn.metrics import accuracy_score,hamming_loss, log_loss, f1_score

In [16]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.210468920393
Hamming accuracy =  0.489099870807
Hamming loss =  0.21654463312042374
Log loss =  64.4788088035
F1 score =  0.432545907511
